In [21]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model, decomposition, datasets
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from fairlearn.metrics import demographic_parity_difference
from fairlearn.reductions import ExponentiatedGradient, DemographicParity
from sklearn.ensemble import AdaBoostClassifier
from fairlearn.reductions import EqualizedOdds
from sklearn.tree import DecisionTreeClassifier
from fairlearn.metrics import equalized_odds_difference

In [22]:
train = pd.read_csv("recid.csv")
num_cols = train.shape[1]
print("Number of columns in the dataset are : ",num_cols)
train.head(5)
print(train.shape)

Number of columns in the dataset are :  53
(7214, 53)


In [23]:
# age,
# c_charge_degree,
# race,
# age_cat,
# score_text,
# sex,
# priors_count,
# days_b_screening_arrest,
# decile_score,
# is_recid,
# two_year_recid,
# c_jail_in,
# c_jail_out

In [24]:
train_new = train[['age', 'c_charge_degree','race','age_cat', 'score_text', 'sex', 'priors_count', 'days_b_screening_arrest'
                              ,'decile_score', 'is_recid','two_year_recid','c_jail_in','c_jail_out']]
print(train_new.shape)
train_new.head(5)

(7214, 13)


,age,c_charge_degree,race,age_cat,score_text,sex,priors_count,days_b_screening_arrest,decile_score,is_recid,two_year_recid,c_jail_in,c_jail_out
0,69,F,Other,Greater than 45,Low,Male,0,-1.0,1,0,0,8/13/2013 6:03,8/14/2013 5:41
1,34,F,African-American,25 - 45,Low,Male,0,-1.0,3,1,1,1/26/2013 3:45,2/5/2013 5:36
2,24,F,African-American,Less than 25,Low,Male,4,-1.0,4,1,1,4/13/2013 4:58,4/14/2013 7:02
3,23,F,African-American,Less than 25,High,Male,1,NaN,8,0,0,NaN,NaN
4,43,F,Other,25 - 45,Low,Male,2,NaN,1,0,0,NaN,NaN


In [25]:
train_new = train_new.dropna()

print(train_new.shape)
train_new.head(5)

(6907, 13)


,age,c_charge_degree,race,age_cat,score_text,sex,priors_count,days_b_screening_arrest,decile_score,is_recid,two_year_recid,c_jail_in,c_jail_out
0,69,F,Other,Greater than 45,Low,Male,0,-1.0,1,0,0,8/13/2013 6:03,8/14/2013 5:41
1,34,F,African-American,25 - 45,Low,Male,0,-1.0,3,1,1,1/26/2013 3:45,2/5/2013 5:36
2,24,F,African-American,Less than 25,Low,Male,4,-1.0,4,1,1,4/13/2013 4:58,4/14/2013 7:02
5,44,M,Other,25 - 45,Low,Male,0,0.0,1,0,0,11/30/2013 4:50,12/1/2013 12:28
6,41,F,Caucasian,25 - 45,Medium,Male,14,-1.0,6,1,1,2/18/2014 5:08,2/24/2014 12:18


In [26]:
train_new = train_new[train_new['is_recid'] != -1]
train_new = train_new[train_new['c_charge_degree'] != "O"]
train_new = train_new[train_new['score_text'] != 'N/A']
train_new = train_new[train_new['days_b_screening_arrest'] <= 30]
train_new = train_new[train_new['days_b_screening_arrest'] >= -30]


print(train_new.shape)
train_new.head(5)

(6172, 13)


,age,c_charge_degree,race,age_cat,score_text,sex,priors_count,days_b_screening_arrest,decile_score,is_recid,two_year_recid,c_jail_in,c_jail_out
0,69,F,Other,Greater than 45,Low,Male,0,-1.0,1,0,0,8/13/2013 6:03,8/14/2013 5:41
1,34,F,African-American,25 - 45,Low,Male,0,-1.0,3,1,1,1/26/2013 3:45,2/5/2013 5:36
2,24,F,African-American,Less than 25,Low,Male,4,-1.0,4,1,1,4/13/2013 4:58,4/14/2013 7:02
5,44,M,Other,25 - 45,Low,Male,0,0.0,1,0,0,11/30/2013 4:50,12/1/2013 12:28
6,41,F,Caucasian,25 - 45,Medium,Male,14,-1.0,6,1,1,2/18/2014 5:08,2/24/2014 12:18


In [27]:
train_new = train_new.drop(['c_jail_in','c_jail_out','is_recid'],axis=1)
print(train_new.shape)
train_new.head(5)

(6172, 10)


,age,c_charge_degree,race,age_cat,score_text,sex,priors_count,days_b_screening_arrest,decile_score,two_year_recid
0,69,F,Other,Greater than 45,Low,Male,0,-1.0,1,0
1,34,F,African-American,25 - 45,Low,Male,0,-1.0,3,1
2,24,F,African-American,Less than 25,Low,Male,4,-1.0,4,1
5,44,M,Other,25 - 45,Low,Male,0,0.0,1,0
6,41,F,Caucasian,25 - 45,Medium,Male,14,-1.0,6,1


In [28]:
# train_new['crime_factor'] = pd.Categorical(train_new['c_charge_degree'])
# train_new['age_factor'] = pd.Categorical(train_new['age_cat'])
# # train_new['age_factor'] = train_new['age_factor'].cat.reorder_categories(
# #     [train_new['age_factor'].cat.categories[0]] + list(train_new['age_factor'].cat.categories[1:]))
# train_new['race_factor '] = pd.Categorical(train_new['race'])
# # train_new['race_factor'] = train_new['race_factor'].cat.reorder_categories(
# #     list(train_new['race_factor'].cat.categories[:2]) + [train_new['race_factor'].cat.categories[2]]
# #     + list(train_new['race_factor'].cat.categories[3:]))
# train_new['gender_factor'] = pd.Categorical(train_new['sex'], categories=['Female', 'Male'], ordered=False)
# # train_new['gender_factor'] = train_new['gender_factor'].cat.reorder_categories(
# #     list(train_new['gender_factor'].cat.categories[1:]) + [train_new['gender_factor'].cat.categories[0]])
# train_new['score_factor'] = pd.Categorical(train_new['score_text'] != 'Low',
#                      categories=[False, True], ordered=False).rename_categories(['LowScore', 'HighScore'])

# print(train_new.shape)
# train_new.head(5)

In [30]:
train_final = pd.get_dummies(train_new)
train_final.columns

Index(['age', 'priors_count', 'days_b_screening_arrest', 'decile_score',
       'two_year_recid', 'c_charge_degree_F', 'c_charge_degree_M',
       'race_African-American', 'race_Asian', 'race_Caucasian',
       'race_Hispanic', 'race_Native American', 'race_Other',
       'age_cat_25 - 45', 'age_cat_Greater than 45', 'age_cat_Less than 25',
       'score_text_High', 'score_text_Low', 'score_text_Medium', 'sex_Female',
       'sex_Male'],
      dtype='object')

In [31]:
train_final = train_final.drop('two_year_recid',axis=1)
X = train_final
Y = train_new['two_year_recid']
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.33,random_state=18)
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (4135, 20)
Shape of X_test: (2037, 20)
Shape of y_train: (4135,)
Shape of y_test: (2037,)


In [57]:
# create a logistic regression object with iterations parameter and gini index parameter
logreg = LogisticRegression(max_iter=1000, solver='lbfgs', penalty='l2', random_state=18, n_jobs=-1, class_weight='balanced')

logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

cm=np.array(confusion_matrix(y_test,y_pred,labels=[1,0]))
confusion=pd.DataFrame(cm,index=['is_recividated','is_not_recividated'],columns=['predicted_recividated','predicted_not_recividated'])
print(confusion)

print(classification_report(y_test,y_pred))

scores = cross_val_score(logreg,X_train,y_train,cv=5)
np.mean(scores)

""" Predicted +ve and are categorized as African American by the race variable"""
X_test[y_pred == 1]['race_African-American'].value_counts()[1]

""" Predicted +ve and are categorized as Caucasian by the race variable"""
X_test[y_pred == 1]['race_Caucasian'].value_counts()[1]

[[tn , fp],[fn , tp]] = confusion_matrix(y_test,y_pred)
print("True Negatives       : ", tn)
print("False Positives      : ", fp)
print("False Negatives      : ", fn)
print("True Positives       : ", tp)

# calculate confusion matrix
cm = confusion_matrix(y_test, y_pred)

# get indices of false positive predictions
fp_indices = np.where((y_pred == 1) & (y_test == 0))[0]

# filter test set to get only false positive predictions
fp_data = X_test.iloc[fp_indices]

# count number of false positives for African-American and Caucasian races
fp_African = fp_data['race_African-American'].sum()
fp_Caucasian = fp_data['race_Caucasian'].sum()

# calculate total number of false positives
fp_total = len(fp_data)

# print results
print("False positive predictions by the model")
print("African-American: ", fp_African)
print("Caucasian: ", fp_Caucasian)
print("Total False Positives: ", fp_total)

print("% of False positives")
print("African-American: %.2f" % (fp_African / fp_total))
print("Caucasian: %.2f" % (fp_Caucasian / fp_total))


# calculate confusion matrix
cm = confusion_matrix(y_test, y_pred)

# get indices of true positive predictions
tp_indices = np.where((y_pred == 1) & (y_test == 1))[0]

# filter test set to get only true positive predictions
tp_data = X_test.iloc[tp_indices]

# count number of true positives for African-American and Caucasian races
tp_African = tp_data['race_African-American'].sum()
tp_Caucasian = tp_data['race_Caucasian'].sum()

# calculate total number of true positives
tp_total = len(tp_data)

# print results
print("True positive predictions by the model")
print("African-American: ", tp_African)
print("Caucasian: ", tp_Caucasian)
print("Total True Positives: ", tp_total)

print("% of True positives")
print("African-American: %.2f" % (tp_African / tp_total))
print("Caucasian: %.2f" % (tp_Caucasian / tp_total))

# get indices of false negative predictions
fn_indices = np.where((y_pred == 0) & (y_test == 1))[0]

# filter test set to get only false negative predictions
fn_data = X_test.iloc[fn_indices]

# count number of false negatives for African-American and Caucasian races
fn_African = fn_data['race_African-American'].sum()
fn_Caucasian = fn_data['race_Caucasian'].sum()

# calculate total number of false negatives
fn_total = len(fn_data)

# print results
print("False negative predictions by the model")
print("African-American: ", fn_African)
print("Caucasian: ", fn_Caucasian)
print("Total False Negatives: ", fn_total)

print("% of False negatives")
print("African-American: %.2f" % (fn_African / (fn_African + tp_African)))
print("Caucasian: %.2f" % (fn_Caucasian / (fn_Caucasian + tp_Caucasian)))


# get indices of true negative predictions
tn_indices = np.where((y_pred == 0) & (y_test == 0))[0]

# filter test set to get only true negative predictions
tn_data = X_test.iloc[tn_indices]

# count number of true negatives for African-American and Caucasian races
tn_African = tn_data['race_African-American'].sum()
tn_Caucasian = tn_data['race_Caucasian'].sum()

# calculate total number of true negatives
tn_total = len(tn_data)

# print results
print("True negative predictions by the model")
print("African-American: ", tn_African)
print("Caucasian: ", tn_Caucasian)
print("Total True Negatives: ", tn_total)

print("% of True negatives")
print("African-American: %.2f" % (tn_African / tn_total))
print("Caucasian: %.2f" % (tn_Caucasian / tn_total))

Accuracy: 0.6774668630338734
                    predicted_recividated  predicted_not_recividated
is_recividated                        598                        351
is_not_recividated                    306                        782
              precision    recall  f1-score   support

           0       0.69      0.72      0.70      1088
           1       0.66      0.63      0.65       949

    accuracy                           0.68      2037
   macro avg       0.68      0.67      0.67      2037
weighted avg       0.68      0.68      0.68      2037

True Negatives       :  782
False Positives      :  306
False Negatives      :  351
True Positives       :  598
False positive predictions by the model
African-American:  201
Caucasian:  72
Total False Positives:  306
% of False positives
African-American: 0.66
Caucasian: 0.24
True positive predictions by the model
African-American:  403
Caucasian:  146
Total True Positives:  598
% of True positives
African-American: 0.67
Caucasian: 

In [49]:
# Define the sensitive features
sensitive_features = np.array([X_train['race_Caucasian'], X_train['race_African-American']]).T
sensitive_features_test = np.array([X_test['race_Caucasian'], X_test['race_African-American']]).T

# Define the fairness constraint
constraint = EqualizedOdds()

# Define the model and the optimizer
logreg = LogisticRegression(max_iter=1000, solver='lbfgs', penalty='l2', random_state=18, n_jobs=-1, class_weight='balanced')

#ForestClassifier(n_estimators=100)
optimizer = ExponentiatedGradient(logreg, constraints=constraint)

# Fit the optimizer to the training data
optimizer.fit(X_train, y_train, sensitive_features=sensitive_features)

# Evaluate the fairness of the model on the test data
y_pred = optimizer.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
cm=np.array(confusion_matrix(y_test,y_pred,labels=[1,0]))
confusion=pd.DataFrame(cm,index=['is_recividated','is_not_recividated'],columns=['predicted_recividated','predicted_not_recividated'])
print(confusion)
dp_diff = demographic_parity_difference(y_test, y_pred, sensitive_features=sensitive_features_test)
eo_diff = equalized_odds_difference(y_test, y_pred, sensitive_features=sensitive_features_test)
print("Demographic parity difference: ", dp_diff)
print("Equalized odds difference: ", eo_diff)
from sklearn.metrics import confusion_matrix

# Compute the confusion matrix for the Caucasian group
cm_caucasian = confusion_matrix(y_test[sensitive_features_test[:, 0] == 1], y_pred[sensitive_features_test[:, 0] == 1])
false_positives_caucasian = cm_caucasian[0, 1]

# Compute the confusion matrix for the African-American group
cm_african_american = confusion_matrix(y_test[sensitive_features_test[:, 1] == 1], y_pred[sensitive_features_test[:, 1] == 1])
false_positives_african_american = cm_african_american[0, 1]

print("False positives for the Caucasian group: ", false_positives_caucasian)
print("False positives for the African-American group: ", false_positives_african_american)

Accuracy: 0.648993618065783
                    predicted_recividated  predicted_not_recividated
is_recividated                        600                        349
is_not_recividated                    366                        722
Demographic parity difference:  0.03174402135425175
Equalized odds difference:  0.04706763844424766
False positives for the Caucasian group:  133
False positives for the African-American group:  169


In [50]:
# Compute the confusion matrix for the Caucasian group
cm_caucasian = confusion_matrix(y_test[sensitive_features_test[:, 0] == 1], y_pred[sensitive_features_test[:, 0] == 1])
false_positives_caucasian = cm_caucasian[0, 1]

# Compute the confusion matrix for the African-American group
cm_african_american = confusion_matrix(y_test[sensitive_features_test[:, 1] == 1], y_pred[sensitive_features_test[:, 1] == 1])
false_positives_african_american = cm_african_american[0, 1]

# Compute the confusion matrix for both groups
cm_both = confusion_matrix(y_test, y_pred)
false_positives_both = cm_both[0, 1] + cm_both[1, 1]

print("False positives for the Caucasian group: ", false_positives_caucasian)
print("False positives for the African-American group: ", false_positives_african_american)
print("False positives for both groups: ", false_positives_both)

# Compute the false positive rate for both groups
total_negatives = cm_both[0, 1] + cm_both[1, 1]
fpr_caucasian = false_positives_caucasian / total_negatives
fpr_african_american = false_positives_african_american / total_negatives
fpr_both = false_positives_both / total_negatives

print("False positive rate for the Caucasian group: ", fpr_caucasian)
print("False positive rate for the African-American group: ", fpr_african_american)
print("False positive rate for both groups: ", fpr_both)

False positives for the Caucasian group:  133
False positives for the African-American group:  169
False positives for both groups:  966
False positive rate for the Caucasian group:  0.13768115942028986
False positive rate for the African-American group:  0.17494824016563146
False positive rate for both groups:  1.0


In [33]:
# create a logistic regression object with iterations parameter and gini index parameter
from sklearn.tree import DecisionTreeClassifier

# create a DecisionTreeClassifier object with Gini index as the splitting criterion
dt = DecisionTreeClassifier(max_depth=1, criterion='gini')

# fit the model to the training data
dt.fit(X_train, y_train)

# make predictions on the test data
y_pred = dt.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

cm=np.array(confusion_matrix(y_test,y_pred,labels=[1,0]))
confusion=pd.DataFrame(cm,index=['is_recividated','is_not_recividated'],columns=['predicted_recividated','predicted_not_recividated'])
print(confusion)

print(classification_report(y_test,y_pred))

scores = cross_val_score(logreg,X_train,y_train,cv=5)
np.mean(scores)

""" Predicted +ve and are categorized as African American by the race variable"""
X_test[y_pred == 1]['race_African-American'].value_counts()[1]

""" Predicted +ve and are categorized as Caucasian by the race variable"""
X_test[y_pred == 1]['race_Caucasian'].value_counts()[1]

[[tn , fp],[fn , tp]] = confusion_matrix(y_test,y_pred)
print("True Negatives       : ", tn)
print("False Positives      : ", fp)
print("False Negatives      : ", fn)
print("True Positives       : ", tp)

# calculate confusion matrix
cm = confusion_matrix(y_test, y_pred)

# get indices of false positive predictions
fp_indices = np.where((y_pred == 1) & (y_test == 0))[0]

# filter test set to get only false positive predictions
fp_data = X_test.iloc[fp_indices]

# count number of false positives for African-American and Caucasian races
fp_African = fp_data['race_African-American'].sum()
fp_Caucasian = fp_data['race_Caucasian'].sum()

# calculate total number of false positives
fp_total = len(fp_data)

# print results
print("False positive predictions by the model")
print("African-American: ", fp_African)
print("Caucasian: ", fp_Caucasian)
print("Total False Positives: ", fp_total)

print("% of False positives")
print("African-American: %.2f" % (fp_African / fp_total))
print("Caucasian: %.2f" % (fp_Caucasian / fp_total))


# calculate confusion matrix
cm = confusion_matrix(y_test, y_pred)

# get indices of true positive predictions
tp_indices = np.where((y_pred == 1) & (y_test == 1))[0]

# filter test set to get only true positive predictions
tp_data = X_test.iloc[tp_indices]

# count number of true positives for African-American and Caucasian races
tp_African = tp_data['race_African-American'].sum()
tp_Caucasian = tp_data['race_Caucasian'].sum()

# calculate total number of true positives
tp_total = len(tp_data)

# print results
print("True positive predictions by the model")
print("African-American: ", tp_African)
print("Caucasian: ", tp_Caucasian)
print("Total True Positives: ", tp_total)

print("% of True positives")
print("African-American: %.2f" % (tp_African / tp_total))
print("Caucasian: %.2f" % (tp_Caucasian / tp_total))

Accuracy: 0.6475208640157094
                    predicted_recividated  predicted_not_recividated
is_recividated                        465                        484
is_not_recividated                    234                        854
              precision    recall  f1-score   support

           0       0.64      0.78      0.70      1088
           1       0.67      0.49      0.56       949

    accuracy                           0.65      2037
   macro avg       0.65      0.64      0.63      2037
weighted avg       0.65      0.65      0.64      2037

True Negatives       :  854
False Positives      :  234
False Negatives      :  484
True Positives       :  465
False positive predictions by the model
African-American:  156
Caucasian:  55
Total False Positives:  234
% of False positives
African-American: 0.67
Caucasian: 0.24
True positive predictions by the model
African-American:  327
Caucasian:  107
Total True Positives:  465
% of True positives
African-American: 0.70
Caucasian: 

In [34]:
rf_classifier = RandomForestClassifier(n_estimators=100)
rf_classifier.fit(X_train, y_train)
y_pred = rf_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

cm=np.array(confusion_matrix(y_test,y_pred,labels=[1,0]))
confusion=pd.DataFrame(cm,index=['is_recividated','is_not_recividated'],columns=['predicted_recividated','predicted_not_recividated'])
print(confusion)

print(classification_report(y_test,y_pred))

scores = cross_val_score(logreg,X_train,y_train,cv=5)
np.mean(scores)

""" Predicted +ve and are categorized as African American by the race variable"""
X_test[y_pred == 1]['race_African-American'].value_counts()[1]

""" Predicted +ve and are categorized as Caucasian by the race variable"""
X_test[y_pred == 1]['race_Caucasian'].value_counts()[1]

[[tn , fp],[fn , tp]] = confusion_matrix(y_test,y_pred)
print("True Negatives       : ", tn)
print("False Positives      : ", fp)
print("False Negatives      : ", fn)
print("True Positives       : ", tp)

# calculate confusion matrix
cm = confusion_matrix(y_test, y_pred)

# get indices of false positive predictions
fp_indices = np.where((y_pred == 1) & (y_test == 0))[0]

# filter test set to get only false positive predictions
fp_data = X_test.iloc[fp_indices]

# count number of false positives for African-American and Caucasian races
fp_African = fp_data['race_African-American'].sum()
fp_Caucasian = fp_data['race_Caucasian'].sum()

# calculate total number of false positives
fp_total = len(fp_data)

# print results
print("False positive predictions by the model")
print("African-American: ", fp_African)
print("Caucasian: ", fp_Caucasian)
print("Total False Positives: ", fp_total)

print("% of False positives")
print("African-American: %.2f" % (fp_African / fp_total))
print("Caucasian: %.2f" % (fp_Caucasian / fp_total))


# calculate confusion matrix
cm = confusion_matrix(y_test, y_pred)

# get indices of true positive predictions
tp_indices = np.where((y_pred == 1) & (y_test == 1))[0]

# filter test set to get only true positive predictions
tp_data = X_test.iloc[tp_indices]

# count number of true positives for African-American and Caucasian races
tp_African = tp_data['race_African-American'].sum()
tp_Caucasian = tp_data['race_Caucasian'].sum()

# calculate total number of true positives
tp_total = len(tp_data)

# print results
print("True positive predictions by the model")
print("African-American: ", tp_African)
print("Caucasian: ", tp_Caucasian)
print("Total True Positives: ", tp_total)

print("% of True positives")
print("African-American: %.2f" % (tp_African / tp_total))
print("Caucasian: %.2f" % (tp_Caucasian / tp_total))

Accuracy: 0.6514482081492391
                    predicted_recividated  predicted_not_recividated
is_recividated                        545                        404
is_not_recividated                    306                        782
              precision    recall  f1-score   support

           0       0.66      0.72      0.69      1088
           1       0.64      0.57      0.61       949

    accuracy                           0.65      2037
   macro avg       0.65      0.65      0.65      2037
weighted avg       0.65      0.65      0.65      2037

True Negatives       :  782
False Positives      :  306
False Negatives      :  404
True Positives       :  545
False positive predictions by the model
African-American:  174
Caucasian:  87
Total False Positives:  306
% of False positives
African-American: 0.57
Caucasian: 0.28
True positive predictions by the model
African-American:  360
Caucasian:  133
Total True Positives:  545
% of True positives
African-American: 0.66
Caucasian: 

In [41]:
base_estimator = DecisionTreeClassifier(criterion='gini')
adaboost = AdaBoostClassifier(estimator=base_estimator, n_estimators=20)
adaboost.fit(X_train, y_train)

# make predictions on the test data
y_pred = adaboost.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

cm=np.array(confusion_matrix(y_test,y_pred,labels=[1,0]))
confusion=pd.DataFrame(cm,index=['is_recividated','is_not_recividated'],columns=['predicted_recividated','predicted_not_recividated'])
print(confusion)

print(classification_report(y_test,y_pred))

scores = cross_val_score(logreg,X_train,y_train,cv=5)
np.mean(scores)

""" Predicted +ve and are categorized as African American by the race variable"""
X_test[y_pred == 1]['race_African-American'].value_counts()[1]

""" Predicted +ve and are categorized as Caucasian by the race variable"""
X_test[y_pred == 1]['race_Caucasian'].value_counts()[1]

[[tn , fp],[fn , tp]] = confusion_matrix(y_test,y_pred)
print("True Negatives       : ", tn)
print("False Positives      : ", fp)
print("False Negatives      : ", fn)
print("True Positives       : ", tp)

# calculate confusion matrix
cm = confusion_matrix(y_test, y_pred)

# get indices of false positive predictions
fp_indices = np.where((y_pred == 1) & (y_test == 0))[0]

# filter test set to get only false positive predictions
fp_data = X_test.iloc[fp_indices]

# count number of false positives for African-American and Caucasian races
fp_African = fp_data['race_African-American'].sum()
fp_Caucasian = fp_data['race_Caucasian'].sum()

# calculate total number of false positives
fp_total = len(fp_data)

# print results
print("False positive predictions by the model")
print("African-American: ", fp_African)
print("Caucasian: ", fp_Caucasian)
print("Total False Positives: ", fp_total)

print("% of False positives")
print("African-American: %.2f" % (fp_African / fp_total))
print("Caucasian: %.2f" % (fp_Caucasian / fp_total))


# calculate confusion matrix
cm = confusion_matrix(y_test, y_pred)

# get indices of true positive predictions
tp_indices = np.where((y_pred == 1) & (y_test == 1))[0]

# filter test set to get only true positive predictions
tp_data = X_test.iloc[tp_indices]

# count number of true positives for African-American and Caucasian races
tp_African = tp_data['race_African-American'].sum()
tp_Caucasian = tp_data['race_Caucasian'].sum()

# calculate total number of true positives
tp_total = len(tp_data)

# print results
print("True positive predictions by the model")
print("African-American: ", tp_African)
print("Caucasian: ", tp_Caucasian)
print("Total True Positives: ", tp_total)

print("% of True positives")
print("African-American: %.2f" % (tp_African / tp_total))
print("Caucasian: %.2f" % (tp_Caucasian / tp_total))

Accuracy: 0.6190476190476191
                    predicted_recividated  predicted_not_recividated
is_recividated                        546                        403
is_not_recividated                    373                        715
              precision    recall  f1-score   support

           0       0.64      0.66      0.65      1088
           1       0.59      0.58      0.58       949

    accuracy                           0.62      2037
   macro avg       0.62      0.62      0.62      2037
weighted avg       0.62      0.62      0.62      2037

True Negatives       :  715
False Positives      :  373
False Negatives      :  403
True Positives       :  546
False positive predictions by the model
African-American:  201
Caucasian:  114
Total False Positives:  373
% of False positives
African-American: 0.54
Caucasian: 0.31
True positive predictions by the model
African-American:  342
Caucasian:  142
Total True Positives:  546
% of True positives
African-American: 0.63
Caucasian:

In [40]:
import xgboost as xgb

# define the base estimator
base_estimator = DecisionTreeClassifier(criterion='gini')

# define the xgboost model
xgb_model = xgb.XGBClassifier(base_estimator=base_estimator, n_estimators=200)

# train the model on the training data
xgb_model.fit(X_train, y_train)

# make predictions on the test data
y_pred = xgb_model.predict(X_test)

# calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# calculate the confusion matrix
cm = np.array(confusion_matrix(y_test, y_pred, labels=[1,0]))
confusion = pd.DataFrame(cm, index=['is_recividated', 'is_not_recividated'], columns=['predicted_recividated', 'predicted_not_recividated'])
print(confusion)

# print the classification report
print(classification_report(y_test, y_pred))

# calculate the cross validation score
scores = cross_val_score(xgb_model, X_train, y_train, cv=5)
np.mean(scores)

[20:38:35] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "base_estimator" } are not used.

Accuracy: 0.6612665684830633
                    predicted_recividated  predicted_not_recividated
is_recividated                        552                        397
is_not_recividated                    293                        795
              precision    recall  f1-score   support

           0       0.67      0.73      0.70      1088
           1       0.65      0.58      0.62       949

    accuracy                           0.66      2037
   macro avg       0.66      0.66      0.66      2037
weighted avg       0.66      0.66      0.66      2037

[20:38:36] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "base_estimator" } are not used.

[20:38:36] WARNING: C:\buildk

0.6532043530834342

In [42]:
from fairlearn.reductions import EqualizedOdds

# Define the sensitive features
sensitive_features = np.array([X_train['race_Caucasian'], X_train['race_African-American']]).T
sensitive_features_test = np.array([X_test['race_Caucasian'], X_test['race_African-American']]).T
print(sensitive_features.shape)
# print(X_train.shape)
# print(y_train.shape)

# Define the fairness constraint
constraint = DemographicParity()

# Define the model and the optimizer
model = LogisticRegression(max_iter=1000, solver='lbfgs', penalty='l2', random_state=18, n_jobs=-1, class_weight='balanced')
optimizer = ExponentiatedGradient(model, constraints=constraint)

#Fit the optimizer to the training data
optimizer.fit(X_train, y_train, sensitive_features=sensitive_features)

# Evaluate the fairness of the model on the test data
y_pred = optimizer.predict(X_test)
print(len(y_pred))
print(y_test.shape)
dp_diff = demographic_parity_difference(y_test, y_pred, sensitive_features=sensitive_features_test)
print("Demographic parity difference: ", dp_diff)

(4135, 2)
2037
(2037,)
Demographic parity difference:  0.015649343574658192


In [58]:
from fairlearn.metrics import equalized_odds_difference

# Define the sensitive features
sensitive_features = np.array([X_train['race_Caucasian'], X_train['race_African-American']]).T
sensitive_features_test = np.array([X_test['race_Caucasian'], X_test['race_African-American']]).T

# Define the fairness constraint
constraint = EqualizedOdds()

# Define the model and the optimizer
model = LogisticRegression(max_iter=1000, solver='lbfgs', penalty='l2', random_state=18, n_jobs=-1, class_weight='balanced')
optimizer = ExponentiatedGradient(model, constraints=constraint)

# Fit the optimizer to the training data
optimizer.fit(X_train, y_train, sensitive_features=sensitive_features)

# Evaluate the fairness of the model on the test data
y_pred = optimizer.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
cm=np.array(confusion_matrix(y_test,y_pred,labels=[1,0]))
confusion=pd.DataFrame(cm,index=['is_recividated','is_not_recividated'],columns=['predicted_recividated','predicted_not_recividated'])
print(confusion)
dp_diff = demographic_parity_difference(y_test, y_pred, sensitive_features=sensitive_features_test)
eo_diff = equalized_odds_difference(y_test, y_pred, sensitive_features=sensitive_features_test)
print("Demographic parity difference: ", dp_diff)
print("Equalized odds difference: ", eo_diff)
from sklearn.metrics import confusion_matrix

# Compute the confusion matrix for the Caucasian group
cm_caucasian = confusion_matrix(y_test[sensitive_features_test[:, 0] == 1], y_pred[sensitive_features_test[:, 0] == 1])
false_positives_caucasian = cm_caucasian[0, 1]

# Compute the confusion matrix for the African-American group
cm_african_american = confusion_matrix(y_test[sensitive_features_test[:, 1] == 1], y_pred[sensitive_features_test[:, 1] == 1])
false_positives_african_american = cm_african_american[0, 1]

print("False positives for the Caucasian group: ", false_positives_caucasian)
print("False positives for the African-American group: ", false_positives_african_american)

Accuracy: 0.6426116838487973
                    predicted_recividated  predicted_not_recividated
is_recividated                        598                        351
is_not_recividated                    377                        711
Demographic parity difference:  0.024560113308274734
Equalized odds difference:  0.06584766584766588
False positives for the Caucasian group:  136
False positives for the African-American group:  169


In [44]:
from sklearn.metrics import confusion_matrix

# Compute the confusion matrix for the Caucasian group
cm_caucasian = confusion_matrix(y_test[sensitive_features_test[:, 0] == 1], y_pred[sensitive_features_test[:, 0] == 1])
tn_caucasian, fp_caucasian, fn_caucasian, tp_caucasian = cm_caucasian.ravel()
false_positive_rate_caucasian = fp_caucasian / (fp_caucasian + tn_caucasian)

# Compute the confusion matrix for the African-American group
cm_african_american = confusion_matrix(y_test[sensitive_features_test[:, 1] == 1], y_pred[sensitive_features_test[:, 1] == 1])
tn_african_american, fp_african_american, fn_african_american, tp_african_american = cm_african_american.ravel()
false_positive_rate_african_american = fp_african_american / (fp_african_american + tn_african_american)

print("False positive rate for the Caucasian group: ", false_positive_rate_caucasian)
print("False positive rate for the African-American group: ", false_positive_rate_african_american)

False positive rate for the Caucasian group:  0.343980343980344
False positive rate for the African-American group:  0.3592814371257485


In [59]:
from sklearn.metrics import confusion_matrix

# Compute the confusion matrix for the Caucasian group
cm_caucasian = confusion_matrix(y_test[sensitive_features_test[:, 0] == 1], y_pred[sensitive_features_test[:, 0] == 1])
true_positives_caucasian = cm_caucasian[1, 1]

# Compute the confusion matrix for the African-American group
cm_african_american = confusion_matrix(y_test[sensitive_features_test[:, 1] == 1], y_pred[sensitive_features_test[:, 1] == 1])
true_positives_african_american = cm_african_american[1, 1]

print("True positives for the Caucasian group: ", true_positives_caucasian)
print("True positives for the African-American group: ", true_positives_african_american)

True positives for the Caucasian group:  186
True positives for the African-American group:  345


In [60]:
from sklearn.metrics import confusion_matrix

# Compute the confusion matrix and FPR for the Caucasian group
cm_caucasian = confusion_matrix(y_test[sensitive_features_test[:, 0] == 1], y_pred[sensitive_features_test[:, 0] == 1])
tn_caucasian, fp_caucasian, fn_caucasian, tp_caucasian = cm_caucasian.ravel()
fpr_caucasian = fp_caucasian / (fp_caucasian + tn_caucasian)

# Compute the confusion matrix and FPR for the African-American group
cm_african_american = confusion_matrix(y_test[sensitive_features_test[:, 1] == 1], y_pred[sensitive_features_test[:, 1] == 1])
tn_african_american, fp_african_american, fn_african_american, tp_african_american = cm_african_american.ravel()
fpr_african_american = fp_african_american / (fp_african_american + tn_african_american)

print("False positive rate for the Caucasian group: ", fpr_caucasian)
print("False positive rate for the African-American group: ", fpr_african_american)

False positive rate for the Caucasian group:  0.33415233415233414
False positive rate for the African-American group:  0.3373253493013972


In [61]:
from sklearn.metrics import confusion_matrix

# Compute the confusion matrix for the Caucasian group
cm_caucasian = confusion_matrix(y_test[sensitive_features_test[:, 0] == 1], y_pred[sensitive_features_test[:, 0] == 1])
false_positives_caucasian = cm_caucasian[0, 1]

# Compute the confusion matrix for the African-American group
cm_african_american = confusion_matrix(y_test[sensitive_features_test[:, 1] == 1], y_pred[sensitive_features_test[:, 1] == 1])
false_positives_african_american = cm_african_american[0, 1]

print("False positives for the Caucasian group: ", false_positives_caucasian)
print("False positives for the African-American group: ", false_positives_african_american)
total_false_positives = false_positives_caucasian + false_positives_african_american
print("Total False positives are: ", total_false_positives)


from sklearn.metrics import confusion_matrix

# Compute the confusion matrix for the Caucasian group
cm_caucasian = confusion_matrix(y_test[sensitive_features_test[:, 0] == 1], y_pred[sensitive_features_test[:, 0] == 1])
true_positives_caucasian = cm_caucasian[1, 1]

# Compute the confusion matrix for the African-American group
cm_african_american = confusion_matrix(y_test[sensitive_features_test[:, 1] == 1], y_pred[sensitive_features_test[:, 1] == 1])
true_positives_african_american = cm_african_american[1, 1]

print("True positives for the Caucasian group: ", true_positives_caucasian)
print("True positives for the African-American group: ", true_positives_african_american)

False positives for the Caucasian group:  136
False positives for the African-American group:  169
Total False positives are:  305
True positives for the Caucasian group:  186
True positives for the African-American group:  345


In [54]:
# Calculate true positive rate for the Caucasian group
tpr_caucasian = tp_caucasian / (tp_caucasian + fn_caucasian)
print(tp_caucasian)
print(fn_caucasian)

# Calculate true positive rate for the African-American group
tpr_african_american = tp_african_american / (tp_african_american + fn_african_american)
print(fn_african_american)

# Calculate the difference in true positive rates
eo_diff = abs(tpr_caucasian - tpr_african_american)

print("Equal opportunity difference: ", eo_diff)
print("True positive rate of caucasian : ",tpr_caucasian) 
print("True positive rate of african american : ",tpr_african_american)

185
104
215
Equal opportunity difference:  0.02822505090377625
True positive rate of caucasian :  0.6401384083044983
True positive rate of african american :  0.6119133574007221
